전처리 중간과정을 진행할 때는 import, 한글깨짐 부분 실행하고 바로 해당 전처리 부분으로 넘어가서 db 가져오면 됨

In [28]:
# 시간 관리 및 데이터 정제
import datetime
import re

# 데이터 관리
import sqlite3
import pandas as pd
import numpy as np

# 텍스트 처리
from konlpy.tag import Hannanum
from nltk import Text
from nltk.tokenize import sent_tokenize, regexp_tokenize

# 시각화
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# 기타
from collections import Counter
import ast
import pickle

In [11]:
# 한글깨짐 해결
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (14,4)

# 마이너스 깨짐 해결
mpl.rcParams['axes.unicode_minus'] = False

# Data Cleaning

## Data Load (raw)

In [4]:
conn = sqlite3.connect('../../data/news_raw.db')
cur = conn.cursor()

In [5]:
query = cur.execute('SELECT * FROM navernews')
cols = [column[0] for column in query.description]
naver = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
naver.shape

(112466, 6)

In [6]:
query = cur.execute('SELECT * FROM daumnews')
cols = [column[0] for column in query.description]
daum = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
daum.shape

(109419, 6)

In [7]:
naver.head()

,id,title,date,body,summary,link
0,1,"이총리 ""폭염, 특별재난 준해…전기요금 제한적 특별배려 검토""",2018.07.31. 오전 11:53,"[연합뉴스TV 제공]국무회의 주재…체계적 폭염대책 주문""최저임금 명암…변화 수용하되...",이낙연 국무총리는 31일 계속되는 폭염으로 전기요금 부담에 대한 우려가 커지고 있는...,https://news.naver.com/main/ranking/read.nhn?r...
1,2,"김성태 ""盧 탄핵때도 기무사 대응문건 의혹…즉시 제출하라""",2018.07.31. 오전 10:25,"""文대통령, 여름휴가 때 노동자·소상공인 생각하길""김성태 자유한국당 원내대표. 뉴스...","김성태 자유한국당 원내대표는 31일 ""지난 2004년 노무현 전 대통령 탄핵 당시 ...",https://news.naver.com/main/ranking/read.nhn?r...
2,3,"합참, 北 ICBM 제작 정황 보도에 ""면밀히 추적·감시중""",2018.07.31. 오전 11:29,"노재천 공보실장 ""공식확인 부적절…한미간 공조""워싱턴포스트 ""액화연료 사용 ICBM...",합동참모본부는 31일 북한이 대륙간탄도미사일을 만들고 있는 정황을 포착했다는 보도와...,https://news.naver.com/main/ranking/read.nhn?r...
3,4,"기무사 ""盧 대통령 탄핵 당시 계엄검토 문건 작성 안해""",2018.07.31. 오후 2:43,김성태 의원 주장에 반박…지난 정부에서도 관련 내용 확인【과천=뉴시스】이영환 기자 ...,국군기무사령부는 31일 노무현 전 대통령 탄핵 당시 계엄령 문건을 작성했다는 김성태...,https://news.naver.com/main/ranking/read.nhn?r...
4,5,"'종전문제 다룰까' 北, 이례적 회담 제의 이어 전향적 태도 보여",2018.07.31. 오후 12:55,"안익산 대표 ""南 흔들어 종전선언? 보도, 그럴 수 있다"" 이해【파주=뉴시스】사진공...",북한이 이례적으로 남북 장성급 군사회담을 먼저 제의한데 이어 회담에서 전향적인 태도...,https://news.naver.com/main/ranking/read.nhn?r...


In [8]:
daum.head()

,id,title,date,body,summary,link
0,1,반복된 음식점 단속에 절망한 26살 청년의 '극단적 선택',2017.08.01. 09:00,"""벌금과 단속 걱정하던 아들, 장사 준비하던 현수막으로 목 메 숨져""결혼까지 미루고...",26살 황승우씨는 지난 30일 오전 경기 남양주시 조안면 막국수집 주방에서 목을 메...,https://v.daum.net/v/20170801090037914
1,2,'즉석만남에 술 한잔 했는데'..휴가철 성범죄 표적된 여성들,2017.08.01. 15:59,#1. 20대 여성 A씨는 지난해 7월 서해 한 해수욕장에 놀러갔다가 우연히 30대...,"본격적인 여름 휴가철을 맞아 해수욕장, 계곡과 같은 피서지에 인파가 몰리면서 성범죄...",https://v.daum.net/v/20170801155924346
2,3,"필리핀서 한국인 관광객 숨진채 발견..""카지노서 거액 잃어""",2017.08.01. 18:41,(하노이=연합뉴스) 김문성 특파원 = 필리핀에서 30대 한국인 관광객이 숨진 채 발...,필리핀에서 30대 한국인 관광객이 숨진 채 발견됐다.\n현지 경찰은 이 한국인이 카...,https://v.daum.net/v/20170801184128126
3,4,"""부인이 공관병 상대 갑질"" 논란 박찬주 육군대장 전역지원(종합)",2017.08.01. 18:50,\n(서울=연합뉴스) 김귀근 이영재 기자 = 공관병에 대한 부인의 '갑질' 의혹을 ...,공관병에 대한 부인의 '갑질' 의혹을 받고 있는 박찬주 육군 2작전사령관(대장)이 ...,https://v.daum.net/v/20170801185020317
4,5,화장실에 몰카 설치해 딸 친구 '찰칵' 목사 입건,2017.08.01. 07:45,(청주=연합뉴스) 이승민 기자 = 청주 흥덕경찰서는 화장실에 '몰래카메라'를 설치해...,경찰에 따르면 A씨는 지난달 29일 오후 3시 25분께 흥덕구의 자신의 집 화장실 ...,https://v.daum.net/v/20170801074552385


In [9]:
cur.close()
conn.close()

In [19]:
naver.isna().sum()

id         0
title      0
date       0
body       0
summary    0
link       0
dtype: int64

In [20]:
daum.isna().sum()

id         0
title      0
date       0
body       0
summary    0
link       0
dtype: int64

## String 길이 250보다 큰 데이터만 남기기 (body)

In [42]:
# 본문의 string 길이가 250보다 큰 데이터만 남기기 => 요약을 위해서 어느정도 길이가 있어야 된다고 판단
naver_copy = naver.loc[naver['body'].str.len() > 250].copy()
daum_copy = daum.loc[daum['body'].str.len() > 250].copy()

naver_copy.reset_index(drop=True, inplace=True)
daum_copy.reset_index(drop=True, inplace=True)

In [43]:
naver.shape, naver_copy.shape, daum.shape, daum_copy.shape

((112466, 6), (111918, 6), (109419, 6), (108033, 6))

## cleaning

In [77]:
def cleaning(text):
    '''
    1. 기자이메일, 각종 계정 제거
    2. 특수문자 대체
    3. 단위 통일
    4. 한자어 대체
    5. 일부 특수 문자, 띄어쓰기, 영어, 한글 등을 제외하고 모두 제거 (ex. 한자)
    6. 언론사명, 기자명 제거
    7. 음성정보 제공용 텍스트 안내문구 제거
    8. 줄바꿈, 중략, 등호 처리
    9. 여러 개의 공백 하나로 줄이기

    '''
    
    # 1. 기자이메일, 각종 계정 제거
    text = re.sub(r'([A-Za-z가-힣]+\s?기자)?\s?[A-Za-z0-9]+@([A-Za-z0-9.]+\s*[A-Za-z가-힣 ]*)?', '', text)
    
    # 2. 특수문자 대체
    text = re.sub(r'＂', '"', text)
    text = re.sub(r'，', ',', text)
    text = re.sub(r'∙', '·', text)
    text = re.sub(r'‘”', '\'"', text)
    text = re.sub(r'……', ' ', text)
    text = re.sub(r'’”', '\'"', text)
    text = re.sub(r'[○ㅇ]', 'ㅇ', text)
    text = re.sub(r'＊', '*', text)
    text = re.sub(r'⋯‘', " '", text)
    text = re.sub(r'…', ' ', text)
    text = re.sub(r'…’', " '", text)
    text = re.sub(r'＆', '&', text)
    text = re.sub(r'“', '"', text)
    text = re.sub(r'”', '"', text)
    text = re.sub(r'(↓)', ' 감소', text)
    text = re.sub(r'(↑)', ' 상승', text)
    text = re.sub(r"\'|\"", ' ', text)
    text = re.sub(r"(▲)", '', text)
    
    # 3. 단위 통일
    text = re.sub(r'(㎥)', '세제곱미터', text)
    text = re.sub(r'(km)', '킬로미터', text)
    text = re.sub(r'(㎞)', '킬로미터', text)
    text = re.sub(r'(mm)', '밀리미터', text)
    text = re.sub(r'(㎜)', '밀리미터', text)
    text = re.sub(r'(ｍ)|(m)', '미터', text)
    text = re.sub(r'(%)', '퍼센트', text)
    text = re.sub(r'(％)', '퍼센트', text)
    text = re.sub(r'(㎿)', '메가와트', text)
    text = re.sub(r'(㏊)', '헥타르', text)
    text = re.sub(r'½', '2분의 1', text)
    text = re.sub(r'㎜', '밀리미터', text)
    text = re.sub(r'mm', '밀리미터', text)
    text = re.sub(r'㎢', '제곱킬로미터', text)
    text = re.sub(r'㎚', '나노미터', text)

    # 4. 한자어 대체
    text = re.sub(r"(美)", '미국', text)
    text = re.sub(r"(佛)", '프랑스', text)
    text = re.sub(r"(UAE)", '아랍에미리트', text)
    text = re.sub(r"(中)", '중국', text)
    text = re.sub(r"(日)", '일본', text)
    text = re.sub(r"(韓)", '한국', text)
    text = re.sub(r"(北)", '북한', text)
    text = re.sub(r"(靑)", '청와대', text)
    text = re.sub(r"(與)", '여당', text)
    text = re.sub(r"(野)", '야당', text)
    text = re.sub(r"(新)", '신규', text)
    text = re.sub(r"(外)", '외', text)
    text = re.sub(r"(英)", '영국', text)
    
    # 5. 일부 특수문자(.,?!/@$%~％·∼()【】©), 띄어쓰기, ASCII 문자(\x00-\x7F), 한글(가-힣)) 제외하고 제거
    text = re.sub(r'[^ .,?!/@$%~％·∼()【】©\x00-\x7F가-힣]+', '', text)
    
    # 6. 언론사명, 기자명 제거
    text = re.sub(r'(취재기자\s?:\s?\w{1,})?', '', text)
    text = re.sub(r'(촬영기자\s?:\s?\w{1,})?', '', text)
    text = re.sub(r'(그래픽\s?:\s?\w{1,})?', '', text)
    text = re.sub(r'(자막뉴스\s?:\s?\w{1,})?', '', text)
    text = re.sub(r'\[앵커\]|\[기자\]', '', text)

        # = 이 있는 경우
    text = re.sub(r'[\[\(【][가-힣A-Za-z0-9\s,\(\)/·!]+=[가-힣A-Za-z0-9\s,\(\)/·]+[\]\)】]\s?([가-힣A-Za-z\s]+=)*',
                   '', text)
        # = 이 없는 경우 본문의 중요 정보가 사라지지 않도록 ()는 match에서 제외
    text = re.sub(r'[\[【][가-힣A-Za-z0-9\s,\(\)/·:|!]+[\]】]\s?([가-힣A-Za-z\s]+=)*', '', text)
    text = re.sub(r'[가-힣A-Za-z0-9\s,\(\)/·:|]*©[가-힣A-Za-z0-9\s,\(\)/·:|]+기자', '', text)
    text = re.sub(r'\(베스트 일레븐\)', '', text)
    text = re.sub(r'News1[가-힣A-Za-z0-9\s,\(\)/·:|]+디자이너', '', text)
    
    # 7. 음성정보 제공용 텍스트 안내문구 제거
    text = re.sub(r'<<[가-힣A-Za-z0-9 ]+>>', '', text)
    
    # 8. 줄바꿈, 중략, 등호 처리
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\.\.\. \(중략\)', ' ', text)
    text = re.sub(r'=', '', text)
    
    # 9. 여러 개의 공백 하나로 줄이기
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [78]:
naver_copy['body'] = naver_copy['body'].apply(cleaning)

In [79]:
daum_copy['body'] = daum_copy['body'].apply(cleaning)

In [80]:
naver_copy['summary'] = naver_copy['summary'].apply(cleaning)

In [81]:
daum_copy['summary'] = daum_copy['summary'].apply(cleaning)

In [82]:
naver_copy.head()

,id,title,date,body,summary,link
0,1,"이총리 ""폭염, 특별재난 준해…전기요금 제한적 특별배려 검토""",2018.07.31. 오전 11:53,국무회의 주재 체계적 폭염대책 주문 최저임금 명암 변화 수용하되 진통 최소화하는 지...,이낙연 국무총리는 31일 계속되는 폭염으로 전기요금 부담에 대한 우려가 커지고 있는...,https://news.naver.com/main/ranking/read.nhn?r...
1,2,"김성태 ""盧 탄핵때도 기무사 대응문건 의혹…즉시 제출하라""",2018.07.31. 오전 10:25,"대통령, 여름휴가 때 노동자·소상공인 생각하길 김성태 자유한국당 원내대표. 김성태 ...",김성태 자유한국당 원내대표는 31일 지난 2004년 노무현 전 대통령 탄핵 당시 국...,https://news.naver.com/main/ranking/read.nhn?r...
2,3,"합참, 北 ICBM 제작 정황 보도에 ""면밀히 추적·감시중""",2018.07.31. 오전 11:29,노재천 공보실장 공식확인 부적절 한미간 공조 워싱턴포스트 액화연료 사용 ICBM 1...,합동참모본부는 31일 북한이 대륙간탄도미사일을 만들고 있는 정황을 포착했다는 보도와...,https://news.naver.com/main/ranking/read.nhn?r...
3,4,"기무사 ""盧 대통령 탄핵 당시 계엄검토 문건 작성 안해""",2018.07.31. 오후 2:43,김성태 의원 주장에 반박 지난 정부에서도 관련 내용 확인 국방부 특별수사단이 계엄령...,국군기무사령부는 31일 노무현 전 대통령 탄핵 당시 계엄령 문건을 작성했다는 김성태...,https://news.naver.com/main/ranking/read.nhn?r...
4,5,"'종전문제 다룰까' 北, 이례적 회담 제의 이어 전향적 태도 보여",2018.07.31. 오후 12:55,"안익산 대표 흔들어 종전선언? 보도, 그럴 수 있다 이해 안익산 북측 수석대표가 3...",북한이 이례적으로 남북 장성급 군사회담을 먼저 제의한데 이어 회담에서 전향적인 태도...,https://news.naver.com/main/ranking/read.nhn?r...


In [87]:
daum_copy.head()

,id,title,date,body,summary,link
0,1,반복된 음식점 단속에 절망한 26살 청년의 '극단적 선택',2017.08.01. 09:00,"벌금과 단속 걱정하던 아들, 장사 준비하던 현수막으로 목 메 숨져 결혼까지 미루고 ...",26살 황승우씨는 지난 30일 오전 경기 남양주시 조안면 막국수집 주방에서 목을 메...,https://v.daum.net/v/20170801090037914
1,2,'즉석만남에 술 한잔 했는데'..휴가철 성범죄 표적된 여성들,2017.08.01. 15:59,#1. 20대 여성 A씨는 지난해 7월 서해 한 해수욕장에 놀러갔다가 우연히 30대...,"본격적인 여름 휴가철을 맞아 해수욕장, 계곡과 같은 피서지에 인파가 몰리면서 성범죄...",https://v.daum.net/v/20170801155924346
2,3,"필리핀서 한국인 관광객 숨진채 발견..""카지노서 거액 잃어""",2017.08.01. 18:41,필리핀에서 30대 한국인 관광객이 숨진 채 발견됐다. 현지 경찰은 이 한국인이 카지...,필리핀에서 30대 한국인 관광객이 숨진 채 발견됐다. 현지 경찰은 이 한국인이 카지...,https://v.daum.net/v/20170801184128126
3,4,"""부인이 공관병 상대 갑질"" 논란 박찬주 육군대장 전역지원(종합)",2017.08.01. 18:50,공관병에 대한 부인의 갑질 의혹을 받고 있는 박찬주 육군 2작전사령관(대장)이 1일...,공관병에 대한 부인의 갑질 의혹을 받고 있는 박찬주 육군 2작전사령관(대장)이 1일...,https://v.daum.net/v/20170801185020317
4,5,화장실에 몰카 설치해 딸 친구 '찰칵' 목사 입건,2017.08.01. 07:45,청주 흥덕경찰서는 화장실에 몰래카메라 를 설치해 한 집에 사는 딸의 친구를 촬영한 ...,경찰에 따르면 A씨는 지난달 29일 오후 3시 25분께 흥덕구의 자신의 집 화장실 ...,https://v.daum.net/v/20170801074552385


## Naver + Daum

이후 학습 과정에서의 처리속도 향상을 위해 각각 최근 5만 건으로 한정

In [89]:
naver_copy['site'] = 'naver'
daum_copy['site'] = 'daum'

In [98]:
naver_cleaned = naver_copy.iloc[61918:].copy()
daum_cleaned = daum_copy.iloc[58033:].copy()

concat

In [99]:
print(naver_cleaned.shape)
print(daum_cleaned.shape)

news_cleaned = pd.concat([naver_cleaned, daum_cleaned], ignore_index=True)

print(news_cleaned.shape)

(50000, 7)
(50000, 7)
(100000, 7)


reset id

In [104]:
news_cleaned.reset_index(inplace=True)
del news_cleaned['id']

In [108]:
news_cleaned.rename({'index':'id'}, inplace=True, axis=1)

In [111]:
news_cleaned

,id,title,date,body,summary,link,site
0,0,"액상담배 美흡연자 8명 숨졌다, 복지부 ""한국 판매 금지 검토""",2019.09.20. 오후 6:38,"중증 폐질환 유발 가능성 있다 미국서 530건 의심환자 발생, 8명 사망국내 상황 ...",정부가 액상형 전자담배 사용 자제를 권고하고 나섰다.복지부는 미국 사례를 참조해 액...,https://news.naver.com/main/ranking/read.nhn?r...,naver
1,1,세력 키우는 태풍 '타파'…22일 밤 부산 앞바다 통과,2019.09.20. 오후 5:19,23일까지 강하고 많은 비제주에 최대 600밀리미터 물폭탄 제17호 태풍 타파가 한...,제17호 태풍 타파 가 한반도에 접근할수록 점점 강해지고 있는 데다 부산 앞바다를 ...,https://news.naver.com/main/ranking/read.nhn?r...,naver
2,2,[내일날씨] 태풍 '타파' 북상…전국에 강한 비바람,2019.09.20. 오전 9:00,강한 비바람 때문에 행인이 우산을 꽉 붙잡고 지나가고 있다. 토요일인 21일 제17...,토요일인 21일 제17호 태풍 타파 영향으로 전국이 흐리고 비가 내리겠다.기상청은 ...,https://news.naver.com/main/ranking/read.nhn?r...,naver
3,3,30년 만에…친정에서 차례 지낸 며느리의 특별한 추석,2019.09.19. 오후 3:01,김현주의 즐거운 갱년기(22) 추석연휴를 맞아 남편과 딸은 제주도로 향했다. 나는 ...,"추석 연휴가 시작되기 전 남편과 딸 아이는 제주도행 비행기를 탔고, 잡지 제작 일정...",https://news.naver.com/main/ranking/read.nhn?r...,naver
4,4,미국 '앨리스 공주'에게 베푼 고종의 오찬…황제의 식탁에 서린 슬픈 역사,2019.09.20. 오후 1:20,1905년 9월20일 고종이 미국 시어도어 루즈벨트 대통령의 딸인 앨리스와의 공식오...,1905년 9월20일 고종 황제는 국빈자격으로 대한제국을 방문한 앨리스 루즈벨트를 ...,https://news.naver.com/main/ranking/read.nhn?r...,naver
...,...,...,...,...,...,...,...
99995,99995,"[부산 프리토크]롯데 허문회 감독 ""전준우 주루 미스라 생각, 마음가짐 좋았다""",2020.08.02. 14:51,롯데 자이언츠 허문회 감독은 전준우의 재치 있는 주루 플레이에 대한 생각을 밝혔다....,롯데 자이언츠 허문회 감독은 전준우의 재치 있는 주루 플레이에 대한 생각을 밝혔다....,https://v.daum.net/v/20200802145136484,daum
99996,99996,"[SC핫피플] 매일 타점 생산하는 러셀, 그가 오자 키움이 살아났다",2020.08.02. 09:28,이 정도면 새 외국인 타자 에디슨 러셀(키움 히어로즈)은 복덩이다. 험난한 7월을 ...,이 정도면 새 외국인 타자 에디슨 러셀(키움 히어로즈)은 복덩이다. 키움은 5월 말...,https://v.daum.net/v/20200802092856893,daum
99997,99997,"STL, 코로나19 확진 4명 추가..선수 총 3명",2020.08.02. 01:16,"세인트루이스 카디널스 구단에 코로나19 확진자가 추가로 발생,총 6명으로 늘어났다....","세인트루이스 카디널스 구단에 코로나19 확진자가 추가로 발생,총 6명으로 늘어났다....",https://v.daum.net/v/20200802011623768,daum
99998,99998,"[사직 게임노트]'노경은 완벽투' 롯데, KIA 8-0 꺾고 '8월 2연승' 신바람",2020.08.02. 19:49,롯데 자이언츠가 8월 첫 주말을 연승으로 장식했다. 롯데는 2일 사직구장에서 열린 ...,롯데 자이언츠가 8월 첫 주말을 연승으로 장식했다. 롯데는 2일 사직구장에서 열린 ...,https://v.daum.net/v/20200802194902874,daum


## DB에 저장 (cleaned)

In [112]:
conn = sqlite3.connect('../../data/news_cleaned.db')
cur = conn.cursor()

In [113]:
news_cleaned.to_sql('news', conn, index=False)

In [114]:
cur.close()
conn.close()